# ReadMe: Script da inserção dos dados para a base de servidores
* Observações Iniciais: Será utilizado o psycopg2 para fazer a conexão com o banco de dados PostgreSQL, recomenda-se fazer de 1 mês por vez.
* É realizado a conexão com o banco de dados e são criados as tabelas oficiais de cadastro, remuneracao e a fato (que é uma junção das duas) e também suas temporárias. Além disso, o script para valores repetidos em cadastro é criado.
* Ideia:
    1. Conexão com o banco de dados.
    * Esses dados serão extraídos e colocados em 'exports'.
    * Depois serão removidos os arquivos não utilizados, onde os que serão utilizados, ficarão na pasta 'processados'.
    * É realizado um ajuste nos csv de processados para se ajustarem melhor a leitura dos dados.
    * São inseridos os valores nas temporárias.
        - As temporárias foram criadas com o propósito de armazenarem os valores e inserirem primeiramente na temp de fato e em seguida adicionar com a oficial.
    * Remoção dos arquivos em 'exports' e 'processados'.
    * Inserção dos valores na fato temporária.
    * Inserção e remoção dos valores da temporária na oficial.
    
    
* Observações: 
 - códigos de órgãos - existem alguns que não são inteiros, então são colocados como varchar
 - Cadastro_all_bruto é a tabela que todos os dados são inseridos. Depois, os dados são atualizados com as foreign keys e por último é inserido sem as duplicações na final tabela (cadastro).

## Observação sobre a Remuneração básica bruta do servidor em reais:
É composta pela soma das parcelas remuneratórias correspondentes ao cargo efetivo, a função ou o cargo comissionado e, ainda, aos seguintes adicionais: adicional de certificação profissional (formação, especialização, aperfeiçoamento, auto estudo), adicional de insalubridade, adicional de periculosidade, adicional noturno, adicional plantão hospitalar, adicional serviço extraordinário, adicional de sobreaviso, adicional de gestão educacional e adicional por tempo de serviço.

In [1]:
import pandas as pd
import numpy as np
import zipfile
import os
import psycopg2
import subprocess
import csv

# Comandos para instalar o psycopg2
* Comando para instalar o psycopg2

conda install -c anaconda psycopg2

### Função Usada 
* Existem casos em remuneracao que a última está preenchida com uma observação, estamos removendo essa observação

In [2]:
def remove_last_line(r):
    count = 0;
    for i in r.columns:
        if pd.isna(r.iloc[-1][i]):
            count = count + 1
    if count > (r.columns.size / 2):
        r = r.drop(r.index[-1])
    return r

## 1. Conexão com o Banco de Dados (Só precisa executar apenas uma vez)

In [3]:
try:
    conn = psycopg2.connect(dbname='pagamento_hibrido_db', user='dmining', host='texas.intranet', password='#@dmining#@')
except:
    print("I am unable to connect to the database")

#### 1.1 É necessário criar uma tabela temporária para os dados

In [4]:
cur = conn.cursor()
cur.execute("""
-- Atuará como uma temporária
CREATE TABLE IF NOT EXISTS servidores.cadastro_all_bruto(
id_servidor_portal bigint, 
  nome varchar, 
  cpf varchar, 
  matricula varchar, 
  descricao_cargo varchar, 
  classe_cargo varchar, 
  referencia_cargo varchar, 
  padrao_cargo varchar, 
  nivel_cargo varchar, 
  sigla_funcao varchar, 
  nivel_funcao varchar, 
  funcao varchar, 
  codigo_atividade varchar, 
  atividade varchar, 
  cod_uorg_lotacao varchar, 
  uorg_lotacao varchar, 
  cod_org_lotacao varchar, 
  org_lotacao varchar, 
  cod_orgsup_lotacao varchar, 
  orgsup_lotacao varchar, 
  cod_uorg_exercicio varchar, 
  uorg_exercicio varchar, 
  cod_org_exercicio varchar, 
  org_exercicio  varchar, 
  cod_orgsup_exercicio varchar, 
  orgsup_exercicio varchar, 
  tipo_vinculo varchar, 
  situacao_vinculo varchar, 
  data_inicio_afastamento varchar, 
  data_termino_afastamento varchar, 
  regime_juridico varchar, 
  jornada_de_trabalho varchar, 
  uf_exercicio varchar
);
""")
conn.commit()

# Script para a extração, pré processamento e inserção dos dados no banco
* Observação: Crie a pasta 'dados' e coloque dentro dela os arquivos .zip

### 2. Extração de .zip para .csv

In [5]:
for file in os.listdir(path='dados/'):
    zip_ref = zipfile.ZipFile('dados/{}'.format(file), 'r')
    zip_ref.extractall('dados/exports')
    zip_ref.close()

### 3. Remoção dos .csv extraídos não utilizados (redução de espaço do disco)

In [6]:
c = 0
for file in os.listdir(path='dados/exports/'):
    if 'Remuneracao.csv' != file.split("_")[1] and 'Cadastro.csv' != file.split("_")[1]:
        os.remove("dados/exports/{}".format(file))
os.mkdir('dados/processados')

### 4. Pré processamento dos .csv para novos .csv com formatos e ajustes

In [7]:
# Esse é um processo um pouco demorado
for file in os.listdir(path='dados/exports/'):
    nome = file.split("_")[1]
    # Para os Cadastros
    if nome == 'Cadastro.csv':
        c = pd.read_csv('dados/exports/{}'.format(file), delimiter=';',encoding='latin1',dtype={'NIVEL_CARGO':object})
        c = c.drop(['OPCAO_PARCIAL', 'DATA_INGRESSO_CARGOFUNCAO', 'DATA_NOMEACAO_CARGOFUNCAO', 
                    'DATA_INGRESSO_ORGAO', 'DOCUMENTO_INGRESSO_SERVICOPUBLICO', 
                    'DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO', 'DIPLOMA_INGRESSO_CARGOFUNCAO',
                    'DIPLOMA_INGRESSO_ORGAO', 'DIPLOMA_INGRESSO_SERVICOPUBLICO'], axis=1)
        c.to_csv('dados/processados/{}'.format(file), sep='\t', index=False)
        
    # Para as Remunerações    
    if nome == 'Remuneracao.csv':
        r = pd.read_csv('dados/exports/{}'.format(file), delimiter=';',encoding='latin1', decimal=',', dtype={'MES':object, 'Id_SERVIDOR_PORTAL':object})
        r = remove_last_line(r);
        r = r[['ANO', 'MES', 'Id_SERVIDOR_PORTAL', 'CPF', 'NOME', 
                 'REMUNERAÇÃO BÁSICA BRUTA (R$)', 'REMUNERAÇÃO BÁSICA BRUTA (U$)']].copy()
        r['DATA'] = r['ANO'].astype(str) + r['MES'].astype(str)
        r.to_csv('dados/processados/{}'.format(file), index=False)
        
    print('Arquivo Finalizado: {}'.format(file));

/home/jose.freires/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Arquivo Finalizado: 20160229_Remuneracao.csv


/home/jose.freires/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Arquivo Finalizado: 20160229_Cadastro.csv


### 5. Inserção dos dados processados para o banco PostgreSQL na tabela temporária

In [8]:
%%time
for file in os.listdir(path='dados/processados/'):
    nome = file.split("_")[1]
    if nome == 'Cadastro.csv': 
        cur = conn.cursor()
        with open('dados/processados/' + file, 'r') as f:
            next(f)  # Skip the header row.
            cur.copy_from(f, 'servidores.cadastro_all_bruto', sep='\t')
        conn.commit()
        print("Importado arquivo de cadastro {} ".format(file))
    if nome == 'Remuneracao.csv':
        cur = conn.cursor()
        with open('dados/processados/' + file, 'r') as f:
            next(f)  # Skip the header row.
            cur.copy_from(f, 'servidores.remuneracao', sep=',')
        conn.commit()
        print("Importado arquivo remuneracao {}".format(file))

Importado arquivo remuneracao 20160229_Remuneracao.csv
Importado arquivo de cadastro 20160229_Cadastro.csv 
CPU times: user 2.4 s, sys: 376 ms, total: 2.78 s
Wall time: 32 s


### 6. Remoção dos arquivos processados e exportados

In [9]:
c = 0
for file in os.listdir(path='dados/exports/'):
        os.remove("dados/exports/{}".format(file))
for file in os.listdir(path='dados/processados/'):
        os.remove("dados/processados/{}".format(file))
os.rmdir("dados/exports")
os.rmdir("dados/processados")

### 7. Inserindo os valores na tabela fato_cargo_funcao_remuneracao e outras tabelas

#### 7.1 Povoando as tabelas de órgãos, cargo e função

In [10]:
cur = conn.cursor()
cur.execute("""
INSERT INTO servidores.orgao_superior
SELECT DISTINCT cod_orgsup_lotacao, orgsup_lotacao
FROM servidores.cadastro_all_bruto;

INSERT INTO servidores.orgao_superior
SELECT DISTINCT cod_orgsup_exercicio, orgsup_exercicio
FROM servidores.cadastro_all_bruto;
""")
conn.commit()
print("Órgão Superior Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO servidores.orgao_subordinado
SELECT DISTINCT cod_org_lotacao, org_lotacao
FROM servidores.cadastro_all_bruto;

INSERT INTO servidores.orgao_subordinado
SELECT DISTINCT cod_org_exercicio, org_exercicio
FROM servidores.cadastro_all_bruto;
""")

conn.commit()
print("Órgão Subordinado Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO servidores.unidade_orgao
SELECT DISTINCT cod_uorg_lotacao, uorg_lotacao
FROM servidores.cadastro_all_bruto;

INSERT INTO servidores.unidade_orgao
SELECT DISTINCT cod_uorg_exercicio, uorg_exercicio
FROM servidores.cadastro_all_bruto;
""")
conn.commit()
print("Unidade Órgão Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO servidores.cargo(descricao_cargo, classe_cargo, referencia_cargo, padrao_cargo, nivel_cargo)
SELECT DISTINCT descricao_cargo, classe_cargo, referencia_cargo, padrao_cargo, nivel_cargo
FROM servidores.cadastro_all_bruto;
""")
conn.commit()
print("Cargo Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO servidores.funcao(sigla_funcao, nivel_funcao, funcao)
SELECT DISTINCT sigla_funcao, nivel_funcao, funcao
FROM servidores.cadastro_all_bruto;
""")
conn.commit()
print("Função Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO servidores.atividade(codigo_atividade, atividade)
SELECT DISTINCT codigo_atividade, atividade
FROM servidores.cadastro_all_bruto;
""")
conn.commit()
print("Atividade Finalizado")

cur = conn.cursor()
cur.execute("""
INSERT INTO servidores.vinculo(tipo_vinculo, situacao_vinculo)
SELECT DISTINCT tipo_vinculo, situacao_vinculo
FROM servidores.cadastro_all_bruto;
""")
conn.commit()
print("Vinculo Finalizado")

Órgão Superior Finalizado
Órgão Subordinado Finalizado
Unidade Órgão Finalizado
Cargo Finalizado
Função Finalizado
Atividade Finalizado
Vinculo Finalizado


#### 7.2 Criando Foreign Keys

In [11]:
cur = conn.cursor()
cur.execute("""
ALTER TABLE servidores.cadastro_all_bruto ADD COLUMN id_cargo integer;
ALTER TABLE servidores.cadastro_all_bruto ADD COLUMN id_funcao integer;
ALTER TABLE servidores.cadastro_all_bruto ADD COLUMN id_atividade integer;
ALTER TABLE servidores.cadastro_all_bruto ADD COLUMN id_vinculo integer;
""")
conn.commit()
print('Novas Colunas para Foreign Keys criado')

Novas Colunas para Foreign Keys criado


#### 7.3 Adicionando a Foreign key para cargo, função, atividade e vinculos

In [12]:
cur = conn.cursor()
cur.execute("""
UPDATE servidores.cadastro_all_bruto
SET id_cargo = cg.id_cargo
FROM servidores.cargo as cg
WHERE cg.descricao_cargo = servidores.cadastro_all_bruto.descricao_cargo AND 
cg.classe_cargo = servidores.cadastro_all_bruto.classe_cargo AND 
cg.referencia_cargo = servidores.cadastro_all_bruto.referencia_cargo AND 
cg.padrao_cargo = servidores.cadastro_all_bruto.padrao_cargo AND 
cg.nivel_cargo = servidores.cadastro_all_bruto.nivel_cargo;
""")
conn.commit()
print('Update Cargo Finalizado')

cur = conn.cursor()
cur.execute("""
UPDATE servidores.cadastro_all_bruto
SET id_funcao = f.id_funcao
FROM servidores.funcao as f
WHERE f.sigla_funcao = servidores.cadastro_all_bruto.sigla_funcao AND 
f.nivel_funcao = servidores.cadastro_all_bruto.nivel_funcao AND f.funcao = servidores.cadastro_all_bruto.funcao;
""")
conn.commit()
print('Update Função Finalizado')

cur = conn.cursor()
cur.execute("""
UPDATE servidores.cadastro_all_bruto
SET id_atividade = a.id_atividade
FROM servidores.atividade as a
WHERE a.codigo_atividade = servidores.cadastro_all_bruto.codigo_atividade AND 
a.atividade = servidores.cadastro_all_bruto.atividade;
""")
conn.commit()
print('Update Atividade Finalizado')

cur = conn.cursor()
cur.execute("""  
UPDATE servidores.cadastro_all_bruto
SET id_vinculo = v.id_vinculo
FROM servidores.vinculo as v
WHERE v.tipo_vinculo = servidores.cadastro_all_bruto.tipo_vinculo AND 
v.situacao_vinculo = servidores.cadastro_all_bruto.situacao_vinculo;
""")
conn.commit()
print('Update Vinculo Finalizado')

Update Cargo Finalizado
Update Função Finalizado
Update Atividade Finalizado
Update Vinculo Finalizado


#### 7.4 Povoando as fato_cargo_funcao_remuneracao

In [13]:
cur = conn.cursor()
cur.execute("""
-- atividade = função (isso veio da versão antiga, ver a issue #157 para mais detalhes e modificar aqui) 
INSERT INTO servidores.fato_cargo_funcao_remuneracao
SELECT 'F' as tipo, c.id_servidor_portal, c.id_cargo, c.id_funcao, 
c.id_atividade, c.id_vinculo, c.cod_uorg_lotacao, c.cod_org_lotacao, c.cod_orgsup_lotacao, 
c.cod_uorg_exercicio, c.cod_org_exercicio, c.cod_orgsup_exercicio, 
r.ano, r.mes, r.remuneracao_basica_bruta_brl, r.remuneracao_basica_bruta_url, r.data
FROM servidores.cadastro_all_bruto c, servidores.remuneracao r
WHERE c.id_servidor_portal = r.id_servidor_portal AND 
(c.codigo_atividade <> '-1' AND c.atividade <> 'Sem informação');
""")
conn.commit()
print("Funções adicionadas na fato")

cur = conn.cursor()
cur.execute("""
INSERT INTO servidores.fato_cargo_funcao_remuneracao
SELECT 'C' as tipo, c.id_servidor_portal, c.id_cargo, c.id_funcao, 
c.id_atividade, c.id_vinculo, c.cod_uorg_lotacao, c.cod_org_lotacao, c.cod_orgsup_lotacao, 
c.cod_uorg_exercicio, c.cod_org_exercicio, c.cod_orgsup_exercicio, 
r.ano, r.mes, r.remuneracao_basica_bruta_brl, r.remuneracao_basica_bruta_url, r.data
FROM servidores.cadastro_all_bruto c, servidores.remuneracao r
WHERE c.id_servidor_portal = r.id_servidor_portal AND 
(c.descricao_cargo <> 'Inválido' AND c.descricao_cargo <> 'Sem informação');
""")
conn.commit()
print("Cargos adicionados na fato")
    
cur = conn.cursor()
cur.execute("""
INSERT INTO servidores.fato_cargo_funcao_remuneracao
SELECT 'N' as tipo, c.id_servidor_portal, c.id_cargo, c.id_funcao, 
c.id_atividade, c.id_vinculo, c.cod_uorg_lotacao, c.cod_org_lotacao, c.cod_orgsup_lotacao, 
c.cod_uorg_exercicio, c.cod_org_exercicio, c.cod_orgsup_exercicio, 
r.ano, r.mes, r.remuneracao_basica_bruta_brl, r.remuneracao_basica_bruta_url, r.data
FROM servidores.cadastro_all_bruto c, servidores.remuneracao r
WHERE c.id_servidor_portal = r.id_servidor_portal AND 
(c.descricao_cargo = 'Inválido' AND c.codigo_atividade = '-1');
""")
conn.commit()
print("Casos especiais adicionados na fato")

Funções adicionadas na fato
Cargos adicionados na fato
Casos especiais adicionados na fato


### 8. Inserção dos valores da temporária na oficial e remoção dos valores duplicados através da trigger

In [14]:
cur = conn.cursor()
cur.execute("""
INSERT INTO servidores.cadastro(
            id_servidor_portal, nome, cpf, matricula, id_cargo, id_funcao, id_atividade, id_vinculo,
            cod_uorg_lotacao, cod_org_lotacao, cod_orgsup_lotacao, cod_uorg_exercicio, cod_org_exercicio, 
            cod_orgsup_exercicio, data_inicio_afastamento, data_termino_afastamento, regime_juridico, 
            jornada_de_trabalho, uf_exercicio)
SELECT id_servidor_portal, nome, cpf, matricula, id_cargo, id_funcao, id_atividade, id_vinculo,
            cod_uorg_lotacao, cod_org_lotacao, cod_orgsup_lotacao, cod_uorg_exercicio, cod_org_exercicio, 
            cod_orgsup_exercicio, data_inicio_afastamento, data_termino_afastamento, regime_juridico, 
            jornada_de_trabalho, uf_exercicio
FROM servidores.cadastro_all_bruto
""")
conn.commit()

In [15]:
cur = conn.cursor()
cur.execute("""
DROP TABLE servidores.cadastro_all_bruto;
""")
conn.commit()